In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Génération de problème d'optimisation

In [2]:
class function:
    def __init__(self, dim, value, grad, hessian):
        self.dim = dim
        self.value = value
        self.grad = grad
        self.hessian = hessian

In [3]:
class probleme:
    def __init__(self, f):
        self.f = f
        
    def __call__(self, x):
        return self.f.value(x)

In [4]:
f_d = {
    "dim": 1,
    "value": lambda x: x[0]**2 - 5 * x[0] + 3,
    "grad": lambda x: np.array([2*x[0] - 5]),
    "hessian": lambda x: np.diag([2])
}
f = function(**f_d)
P = probleme(f)

In [5]:
f_2_d = {
    "dim": 2,
    "value": lambda x: x[0]**2 + 20 * x[1]**2,
    "grad": lambda x: np.array([2*x[0], 40*x[1]]),
    "hessian": lambda x: np.diag([2, 40])
}

f_2 = function(**f_2_d)

# Méthode de Newton

In [6]:
def backtracking(f, x, alpha=0.1, beta=0.8):
    t = 1
    desc_d = -1 * f.grad(x)
    while f.value(x + desc_d * t) > f.value(x) + alpha * t * np.dot(f.grad(x).T, desc_d):
        t = beta * t
    return t, desc_d

In [7]:
def constant(*args):
    return 0.01, 0

In [8]:
class Newton:
    def __init__(self, f, pas, epsilon=0.01):
        self.epsilon = epsilon
        self.f = f
        self.pas = pas
        self.save = np.array([])
        self.dirs = np.array([])
        
    def __call__(self, x0):
        self.save = []
        self.dirs = []
        x = x0
        self.save.append(x)
        dxN = -1 * np.dot(np.linalg.inv(self.f.hessian(x)), self.f.grad(x))
        lmd = -1 * np.dot(self.f.grad(x).T, dxN)
        while lmd / 2 > self.epsilon:
            dxN = -1 * np.dot(np.linalg.inv(self.f.hessian(x)), self.f.grad(x))
            lmd = -1 * np.dot(self.f.grad(x).T, dxN)
            (t, dire) = self.pas(self.f, x)
            self.dirs.append(dire)
            x = x + t * dxN
            self.save.append(x)
        self.save = np.array(self.save)
        self.dirs = np.array(self.dirs)
        return x
    
    def plot(self):
        if self.save.shape[0] == 0:
            raise Exception("The Newton method algorithm has not been run")
        if self.f.dim == 1:
            plt.figure(figsize=(15, 15))
            x = np.linspace(-1 * self.save.max() - 5, self.save.max() + 5, 1000).reshape((1, -1))
            plt.plot(x.reshape((-1)), self.f.value(x))
            plt.scatter(self.save[:, 0], self.f.value(self.save[:, 0].reshape(1, -1)), 50, c="red")
            plt.grid()
            plt.show()
        elif self.f.dim == 2:
            plt.figure(figsize=(15, 15))
            x, y = np.linspace(-1 * self.save[:, 0].max() - 5, self.save[:, 0].max() + 5, 200), np.linspace(- 1 * self.save[:, 1].max() - 5, self.save[:, 1].max() + 5, 200)
            X, Y = np.meshgrid(x, y)
            x_y = np.vstack([X.reshape(1, -1), Y.reshape(1, -1)]).reshape(2, -1)
            plt.contour(X, Y, self.f.value(x_y).reshape(200, -1), 15)
            plt.scatter(self.save[:, 0], self.save[:, 1], 50, c="red")
            plt.grid()
            plt.show()
        else:
            raise Exception("Dimension > 2 not implemented")

In [10]:
meth = Newton(f_2, backtracking)
meth(np.array([100, 100]))
print(meth.dirs)
#meth.plot()

[[-2.00000000e+02 -4.00000000e+03]
 [-1.91203907e+02 -3.82407814e+03]
 [-1.82794670e+02 -3.65589340e+03]
 [-1.74755276e+02 -3.49510551e+03]
 [-1.67069457e+02 -3.34138915e+03]
 [-1.59721665e+02 -3.19443330e+03]
 [-1.52697032e+02 -3.05394064e+03]
 [-1.45981345e+02 -2.91962691e+03]
 [-1.39561018e+02 -2.79122036e+03]
 [-1.33423059e+02 -2.66846119e+03]
 [-1.27555051e+02 -2.55110102e+03]
 [-1.21945121e+02 -2.43890241e+03]
 [-1.16581918e+02 -2.33163835e+03]
 [-1.11454591e+02 -2.22909181e+03]
 [-1.06552766e+02 -2.13105532e+03]
 [-1.01866526e+02 -2.03733051e+03]
 [-9.73863885e+01 -1.94772777e+03]
 [-9.31032898e+01 -1.86206580e+03]
 [-8.90085638e+01 -1.78017128e+03]
 [-8.50939258e+01 -1.70187852e+03]
 [-8.13514553e+01 -1.62702911e+03]
 [-7.77735805e+01 -1.55547161e+03]
 [-7.43530623e+01 -1.48706125e+03]
 [-7.10829800e+01 -1.42165960e+03]
 [-6.79567175e+01 -1.35913435e+03]
 [-6.49679494e+01 -1.29935899e+03]
 [-6.21106288e+01 -1.24221258e+03]
 [-5.93789745e+01 -1.18757949e+03]
 [-5.67674595e+01 -1